In [165]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import glob
import corner, emcee, tqdm
from tweezepy import AV, PSD, MLE
import os

In [166]:
## Opens the folder and stores the data within the files into a Pandas Dataframe
# Requests the folder location from the user
folderPath = input("Type the path to the folder location:")

# Checks to see if the path is correct and returns an error if it is not
if os.path.exists(folderPath):
    print("Using: " + folderPath)
else: 
    raise FileNotFoundError('No such file or directory.')

# Requests the number of reference and experimental beads from the user to create list of the bead names
numRef = int(input("Number of reference beads:"))
numExp = int(input("Number of experimental beads:"))

# Initialization of that list
nameList = ["inf","linearMotor","rotationalMotor"]

# Adds names to the list depending on how many beads were given
for i in range(numRef):
    nameList = nameList + ["xRefBead"+str(i+1), "yRefBead"+str(i+1), "zRefBead"+str(i+1)]
for i in range(numExp): 
    nameList = nameList + ["xExpBead"+str(i+1), "yExpBead"+str(i+1), "zExpBead"+str(i+1)]

# Stores the file names into a list
files = glob.glob(folderPath + '/*.txt')

# Turns the data into a table, then concatenates this table into a Pandas dataframe 
dataTable = [pd.read_table(files[i], names=nameList) for i in range(len(files))]
dataFrame = pd.concat([pd.DataFrame(dataTable[i], columns=nameList) for i in range(len(files))],
          ignore_index=True)

# Uses the conversion values to convert the pixels to nanometers. Also converts the z-axis to nm from um
xConv = 64.6
yConv = 64.7
for column in dataFrame.columns:
    if column[0] == 'x': dataFrame[column] = dataFrame[column]*xConv
    if column[0] == 'y': dataFrame[column] = dataFrame[column]*yConv
    if column[0] == 'z': dataFrame[column] = dataFrame[column]*1000

# Requests the fps and then creates a time series from this
fps = int(input("Frames per second:"))
timeVector = pd.Series(range(len(dataFrame)))/400

Using: C:\Users\maddr\OneDrive\Documents\MTCode\Magnetic-Tweezers\Data\October\6\1\dataTSV


In [183]:
# lowForceStart = 40000
# lowForceEnd = 60000
lowForce = []
highForce = []
mediumForce = []
for index, row in dataFrame.iterrows():
    if row['linearMotor'] < 15:
        lowForce = lowForce + [index]
    if row['linearMotor'] > 15 & row['linearMotor'] < 28:
        highForce = highForce + [index]

In [186]:

plt.plot()

64013

In [168]:
timeVector

0           0.0000
1           0.0025
2           0.0050
3           0.0075
4           0.0100
            ...   
203608    509.0200
203609    509.0225
203610    509.0250
203611    509.0275
203612    509.0300
Length: 203613, dtype: float64